Santiago León Salinas			| A01734958

Josue Taboada Elvira		            | A01732454

Omar Jiménez				| A01732097

Alan Eduardo Dávila			| A01731445

Juan Carlos Llanos		            | A01734916

# **1.- Preprocesamiento**

Carga de datos

In [2]:
#Importación de librerías
import pandas as pd
from pandas import Series,DataFrame
import math
from sklearn.model_selection import train_test_split
import nltk
!pip install -q wordcloud
import wordcloud
from nltk.tokenize import word_tokenize,wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

#Extraer los datos de csv
setInicialFake=pd.read_csv('Fake.csv')
setInicialTrue=pd.read_csv('True.csv')

# Agregar una columna 'label' para marcar los valores
setInicialFake['clasificacion'] = 0  # Marcar como 0 para fake
setInicialTrue['clasificacion'] = 1  # Marcar como 1 para true

setFake_sampled = setInicialFake.sample(n=3000, random_state=42)
setTrue_sampled = setInicialTrue.sample(n=7000, random_state=42)
#Unir los conjuntos
merged_data = pd.concat([setFake_sampled, setTrue_sampled], ignore_index=True)
# Mezclar los datos aleatoriamente
merged_data = merged_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Dividir los datos en conjuntos de entrenamiento y prueba
setInicial_tin, setInicial_tst = train_test_split(merged_data, test_size=0.3, random_state=42)

/bin/bash: /mnt/c/Users/chito/Documents/Python/Act2.2/venv/bin/pip: /mnt/c/Users/chito/Documents/Python/Act2_2/venv/bin/python3: bad interpreter: No such file or directory


[nltk_data] Downloading package punkt to /home/sprk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sprk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Análisis de texto

In [3]:

########## Tokenización #########

vocabulary = set()

for index, row in setInicial_tin.iterrows():
    # Tokenizar el título y el texto, y agregar las palabras únicas al vocabulario
    title_tokens = word_tokenize(row['title'])
    text_tokens = word_tokenize(row['text'])
    vocabulary.update(title_tokens)
    vocabulary.update(text_tokens)

# Obtener el tamaño del vocabulario
tamanio_vocabulario = len(vocabulary)
print("Tamaño del vocabulario:", tamanio_vocabulario)

############ Lematizador ###############

lemmatizer = WordNetLemmatizer()

def extraer_lemas(txt):
    tokens = txt
    lemas = [lemmatizer.lemmatize(token) for token in tokens]
    return lemas

lemas = extraer_lemas(vocabulary)
print(lemas)

#Extraer lemas:
lista_filtrada = list(filter(lambda palabra: palabra not in lemas, vocabulary))

tamanio_filtrado = len(lista_filtrada)
print("Tamaño del vocabulario (lista_filtrada) después de extraer lexemas:", tamanio_filtrado)

Tamaño del vocabulario: 84447
['Marcin', 'outflanked', 'Falcons', 'chronicle', 'Madbury', 'spill', 'STAFFERS', 'checkered', 'shudder', 'Sambisa', 'Burgess', '21,076', 'Sloppy', 'split', "'good", 'US.But', 'weaker', 'Enabling', 'Cue', 'accomplishing', 'Montclair', 'extorted', 'cost.', 'pic.twitter.com/RTieC8dlKC', 'cafeteria', 'Ex-Utah', 'monetarily', 'obstructionist', 'Reds', '15-years', 'Gadgets', 'mending', 'Tamil', 'demarche', 'Kerk', 'promise.', 'sensation', 'Raju', 'HillarysEarPiece', 'Ulgen', 'agreed.He', 'Jewellery', 'Nawaz', 'transporting', 'raged', 'Honoris', 'WND.Dick', 'access', 'firstly', 'Gillibrand', 'Hampshire.Past', 'commentary', 'disclosure', 'lag', 'Saffir-Simpson', 'Harley-Davidson', 'Schizophrenia', 'four-percent', 'Chutiwong', '14-year', 'smaller', 'deals.The', 'Castile', 're-articulating', 'APWLD', 'typically', 'Survey', '102', 'week.The', 'Watergate.Seriously', 'eon', 'outing', 'Kespry', 'Ariz.', 'searches.Despite', 'cohesive', 'safe.The', 'pre-primary', 'excerpt

Analizar la pertinencia de eliminar o no las palabras cerradas, incluir sus conclusiones

# **2.- Entrenamiento y clasificación**

## 2.1.- Representación _tf-idf_ de los documentos

### Frecuencia de términos (_tf_)

In [6]:
import numpy as np

def representacion_tf(dataset):
    reptf = pd.DataFrame(np.zeros((len(dataset), len(lista_filtrada))), columns=lista_filtrada)
    for (index_train, row_train), (index_vocab, row_vocab) in zip(dataset.iterrows(), reptf.iterrows()):
        texto = word_tokenize(row_train.title + " " + row_train.text)
        for palabra in texto:
            if palabra in reptf.columns:
                reptf.at[index_vocab, palabra] = reptf.at[index_vocab, palabra] + 1.0
        reptf.at[index_vocab, 'clasificacion'] = dataset.at[index_train, 'clasificacion']
    
    return reptf


vocabulariotf = representacion_tf(setInicial_tin)

clasificacion = vocabulariotf['clasificacion']
 
vocabulariotf = vocabulariotf.drop(columns=['clasificacion']).loc[:, (vocabulariotf.drop(columns=['clasificacion']).max(axis=0) != 1)]

vocabulariotf['clasificacion'] = clasificacion

print(len(vocabulariotf.columns))
print(vocabulariotf)

vocabulario_filtrado_cantidad = vocabulariotf.columns[:-1]

print(vocabulario_filtrado_cantidad)

        

2030
      spills  disclosures  backers  provinces  buses  pensions  platforms  \
0        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
1        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
2        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
3        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
4        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
...      ...          ...      ...        ...    ...       ...        ...   
6995     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6996     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6997     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6998     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6999     0.0          0.0      0.0        0.0    0.0       0.0        0.0   

      portfolios  crimes  knives  ...  costs  innocents  plans  hikes 

### Frecuencia inversa del documento (_idf_)

In [7]:
def representacion_idf(reptf):
    repidf = pd.DataFrame(np.zeros((1, len(vocabulario_filtrado_cantidad))), columns=vocabulario_filtrado_cantidad)
    for column in vocabulario_filtrado_cantidad:
        non_zero_count = (reptf[column] != 0).sum()
        if non_zero_count > 0:
            idf = np.log(len(vocabulario_filtrado_cantidad)/non_zero_count)
            repidf.at[0, column] = idf
    
    return repidf

vocabularioidf = representacion_idf(vocabulariotf)
print(vocabularioidf)

     spills  disclosures   backers  provinces    buses  pensions  platforms  \
0  5.823539     3.926419  3.537761   3.951737  4.84271  4.181311   4.396423   

   portfolios    crimes    knives  ...   tariffs    costs  innocents  \
0    5.823539  2.422341  4.907248  ...  3.645006  2.16426   7.615298   

      plans     hikes    crises  incidents    blocks  averages  pretends  
0  1.195303  4.570776  3.723478   3.126662  3.901726  6.229004  5.669388  

[1 rows x 2029 columns]


### Ponderación _tf_ - _idf_

In [8]:
def representacion_tf_idf(dataset, reptf, repidf):
    reptf_idf = pd.DataFrame(np.zeros((len(dataset), len(reptf.columns))), columns=reptf.columns)
    for index_vocab, row_vocab in reptf_idf.iterrows():
        for column in reptf_idf.columns[:-1]:
            reptf_idf.at[index_vocab, column] = reptf.at[index_vocab, column] * repidf.at[0, column]
        reptf_idf.at[index_vocab, 'clasificacion'] = reptf.at[index_vocab, 'clasificacion']
    
    return reptf_idf

vocabulariotfidf = representacion_tf_idf(setInicial_tin, vocabulariotf, vocabularioidf)

In [9]:
print(vocabulariotfidf)

      spills  disclosures  backers  provinces  buses  pensions  platforms  \
0        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
1        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
2        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
3        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
4        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
...      ...          ...      ...        ...    ...       ...        ...   
6995     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6996     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6997     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6998     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6999     0.0          0.0      0.0        0.0    0.0       0.0        0.0   

      portfolios    crimes  knives  ...  costs  innocents     plans  hikes 

## 2.2.- Representación binaria o conteo de los documentos

In [11]:
def representacion_binaria(dataset, reptf):
    repbinaria = pd.DataFrame(np.zeros((len(dataset), len(reptf.columns))), columns=reptf.columns)
    for index_vocab, row_vocab in repbinaria.iterrows():
        for column in repbinaria.columns[:-1]:
            if reptf.at[index_vocab, column] > 0:
                repbinaria.at[index_vocab, column] = 1
        repbinaria.at[index_vocab, 'clasificacion'] = reptf.at[index_vocab, 'clasificacion']

    return repbinaria

vocabulariobinario = representacion_binaria(setInicial_tin, vocabulariotf)

print(vocabulariobinario)


      spills  disclosures  backers  provinces  buses  pensions  platforms  \
0        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
1        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
2        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
3        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
4        0.0          0.0      0.0        0.0    0.0       0.0        0.0   
...      ...          ...      ...        ...    ...       ...        ...   
6995     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6996     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6997     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6998     0.0          0.0      0.0        0.0    0.0       0.0        0.0   
6999     0.0          0.0      0.0        0.0    0.0       0.0        0.0   

      portfolios  crimes  knives  ...  costs  innocents  plans  hikes  cris

## 2.3.- Algoritmo de clasificación (Árbol de Decisión)

### Empleando representación _tf-idf_

In [ ]:
from sklearn.tree import DecisionTreeClassifier

################## Clasificador ######################


################## Modelo ###########################
datostfidf = vocabulariotfidf[vocabulariotfidf.columns[:-1]]
clasestfidf = vocabulariotfidf[vocabulariotfidf.columns[-1]]

prof = [5,10,15,20,25]

Train = []
Test = []

for x in prof:
    clasificador = DecisionTreeClassifier(criterion='entropy', max_depth=x)
    modelo = clasificador.fit(datostfidf, clasestfidf)
    predict = modelo.predict()
    

### Empleando representación binaria

In [ ]:
from sklearn.tree import DecisionTreeClassifier

################## Clasificador ######################
clasificador = DecisionTreeClassifier(criterion='entropy', max_depth=5)

################## Modelo ###########################
datostfidf = vocabulariobinario[vocabulariobinario.columns[:-1]]
clasestfidf = vocabulariobinario[vocabulariobinario.columns[-1]]
modelo = clasificador.fit(datostfidf, clasestfidf)

from sklearn.tree import export_text

arbol = export_text(modelo, feature_names=datostfidf.columns.to_list())
print(arbol)



import matplotlib.pyplot as plt
from sklearn import tree
plt.figure(figsize=(10,10))
tree.plot_tree(clasificador,feature_names=datostfidf.columns.to_list(),class_names=['tested_negative','tested_positive'],filled=True)#, rounded=True)
plt.show()

# Conclusiones

Es probable que existan diferencias significativas entre los resultados de cada representación. El TF-IDF otorga más peso a las palabras raras en el corpus pero frecuentes en un documento específico, mientras que la representación de conteo simplemente cuenta la frecuencia de ocurrencia de cada palabra. Estas diferencias pueden afectar el rendimiento del modelo de clasificación.

Los experimentos pueden ubicarse en distintos cuadrantes del diagrama según el comportamiento de las métricas de precisión y exhaustividad. Idealmente, deseamos que los experimentos estén en el cuadrante superior derecho, indicando altos valores de precisión y exhaustividad. Sin embargo, algunos experimentos pueden estar en otros cuadrantes, señalando un desequilibrio entre precisión y exhaustividad.

Es posible identificar palabras que aparecen más frecuentemente en noticias falsas que en noticias reales mediante técnicas de análisis de texto como TF-IDF o la matriz de conteo de palabras. Estas palabras pueden ser indicadores importantes para detectar noticias falsas, aunque es importante considerar que algunas palabras pueden ser comunes en ambos tipos de noticias.

La matriz de confusión es útil para profundizar en el análisis del rendimiento del modelo de clasificación. Además de calcular métricas como precisión, exhaustividad y F1-score, la matriz de confusión permite visualizar cómo el modelo clasifica las instancias en cada clase y cómo se distribuyen los errores de clasificación. Esto puede brindar información valiosa sobre las fortalezas y debilidades del modelo, así como áreas específicas que podrían necesitar más atención durante el ajuste del modelo.

In [ ]:
%%bash
jupyter nbconvert --to html Act2_2_AnalisisdeTexto.ipynb